In [32]:

 from google.colab import files
files.upload()



Saving headphone.csv to headphone (3).csv


{'headphone (3).csv': b'ReviewBody,ReviewStar\n"No doubt it has a great bass and to a great extent noise cancellation and decent sound clarity and mindblowing battery but the following dissapointed me though i tried a lot to adjust.1.Bluetooth range not more than 10m2. Pain in ear due the conical buds(can be removed)3. Wires are a bit long which makes it odd in front.4. No pouch provided.5. Worst part is very low quality and distoring mic. Other person keeps complaining about my voice.\n",3\n"This  earphones are unreliable, i bought it before 15 days meanwhile right side ear buds got cracked automatically and it got divided in two parts, and sound quality is also not that much good but ok, one more thing bass is not good as it is a boat earphones.Guys,Also for the proof i have attached picsPlease see and think before buying\xe2\x80\x8b this unreliable product.Thanks.\n",1\n"i bought itfor 999,I purchased it second time, gifted first one to brother. This is really good.Sound quality is 

In [33]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [34]:
 import pandas as pd
import numpy as np
import string
from collections import Counter, defaultdict

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import sentiwordnet as swn




In [35]:

df = pd.read_csv("headphone.csv")

df.head()



,ReviewBody,ReviewStar
0,No doubt it has a great bass and to a great ex...,3
1,"This earphones are unreliable, i bought it be...",1
2,"i bought itfor 999,I purchased it second time,...",4
3,Its sound quality is adorable. overall it was ...,1
4,Its Awesome... Good sound quality & 8-9 hrs ba...,5


In [36]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

df['tokens'] = df['ReviewBody'].astype(str).apply(clean_text)
df[['ReviewBody', 'tokens']].head()


,ReviewBody,tokens
0,No doubt it has a great bass and to a great ex...,"[doubt, great, bass, great, extent, noise, can..."
1,"This earphones are unreliable, i bought it be...","[earphones, unreliable, bought, 15, days, mean..."
2,"i bought itfor 999,I purchased it second time,...","[bought, itfor, 999i, purchased, second, time,..."
3,Its sound quality is adorable. overall it was ...,"[sound, quality, adorable, overall, good, 2, w..."
4,Its Awesome... Good sound quality & 8-9 hrs ba...,"[awesome, good, sound, quality, 89, hrs, batte..."


In [38]:
def rake(tokens):
    phrases = []
    phrase = []

    for word in tokens:
        if word in stop_words:
            if phrase:
                phrases.append(phrase)
                phrase = []
        else:
            phrase.append(word)

    if phrase:
        phrases.append(phrase)

    freq = Counter()
    degree = Counter()

    for p in phrases:
        length = len(p)
        for w in p:
            freq[w] += 1
            degree[w] += length

    word_score = {w: degree[w] / freq[w] for w in freq}

    phrase_scores = {}
    for p in phrases:
        phrase_scores[' '.join(p)] = sum(word_score[w] for w in p)

    return phrase_scores


In [39]:
aspects = [
    'sound',
    'bass',
    'noise cancellation',
    'battery life',
    'build',
    'price'
]

def extract_adjectives(tokens):
    tagged = pos_tag(tokens)
    return [word for word, tag in tagged if tag.startswith('JJ')]


In [40]:
negations = {'not', 'no', 'never'}

def apply_negation(tokens):
    result = []
    negate = False

    for word in tokens:
        if word in negations:
            negate = True
        elif negate:
            result.append('NEG_' + word)
            negate = False
        else:
            result.append(word)

    return result


In [41]:
def adjective_sentiment(adj):
    synsets = list(swn.senti_synsets(adj, 'a'))
    if not synsets:
        return 0
    return np.mean([s.pos_score() - s.neg_score() for s in synsets])




In [42]:
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [43]:
aspect_sentiments = defaultdict(list)

for _, row in df.iterrows():
    tokens = apply_negation(row['tokens'])
    adjectives = extract_adjectives(tokens)
    text = ' '.join(tokens)

    for aspect in aspects:
        if aspect in text:
            for adj in adjectives:
                base_adj = adj.replace('NEG_', '')
                score = adjective_sentiment(base_adj)

                if adj.startswith('NEG_'):
                    score *= -1

                aspect_sentiments[aspect].append(score)

final_aspect_sentiment = {
    aspect: np.mean(scores) if scores else 0
    for aspect, scores in aspect_sentiments.items()
}

final_aspect_sentiment


{'sound': np.float64(0.22236912611123255),
 'bass': np.float64(0.2054530921820066),
 'noise cancellation': np.float64(0.20131807960565754),
 'battery life': np.float64(0.22947434968344094),
 'price': np.float64(0.22765503080970687),
 'build': np.float64(0.173786088749415)}